# Logistic Regression

- Reminder
- Computing Hypothesis
- Computing Cost Fucntion
- Evaluataion
- Higher Implementation

### Reminder : Logistic Regression

Logistic Regression은 binary classification 문제입니다.. 즉 우리에게 m개의 셈플을 가지고 d의 디맨션을 가진 x가 있다고 생각해 봅니다. 
그러면 (1,d) vector가 m개의 sample이 들어있다고도 볼수 있습니다.   

[  
[7,8,8,9,5,4],  
[3,8,8,6,5,4],  
[6,6,7,9,7,4],  
[3,8,8,6,5,4],  
[1,8,8,6,5,4],  
]                  
과 같은 matrix가 
(m,1)크기의 어떠한
[  
[0],  
[1],  
[1],  
[0],  
[1]  
]   
에 대응된다고 생각하면 됩니다. 
즉 d차원의 (1,d)Vector가 주어졌을때 0또는 1 어디에 가까운지 구하는것이 Logistic Regression 입니다.  
여기서 어떤값이 1일 확률 P(x=1)은 1-P(x=0) 이 되는것은 자명합니다.  

자 이제 우리가 구할 W(Weight Parmeter)를 살펴보겠습니다.  
W는 (d,1)크기의 Vector가 됩니다.  


그리고 마지막으로 우리는 WX를 곱한값에 sigmoid함수를 사용해 0,1과 근사하게 합니다. 

In [7]:
import torch

X = torch.FloatTensor([  
[7,8,8,9,5,4],  
[3,8,8,6,5,4],  
[6,6,7,9,7,4],  
[3,8,8,6,5,4],  
[1,8,8,6,5,4],  
])

Y = torch.FloatTensor([  
[0],  
[1],  
[1],  
[0],  
[1]  
])

# 이제 mul(X,W) = Y 에 대응하는 W 를 찾는것이 목적

위에서 한 예제들을 수식으로 살펴 보도록 하겠습니다.

### Hypothesis

H(X) = 1/1+sigmoid(-W.T*X)  
= P(X=1;W) # W가 주어졌을때 X=1일 값  
= 1-P(X=0;W) 로도 표현 가능합니다.

### Cost

-1/M * mean(y*log(H(X)) + (1-y)(log(1-H(X))

- if y ~= H(x), cost is near0
- if y not H(x), cost is high

### Weight Update via Gradient Descent
W := W -a(d/dW)*cost(W)

a : Learning rate

***

### 코드시작

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [9]:
# For repoducibility
torch.manual_seed(1)

In [10]:
# Training Data
x_data = torch.FloatTensor([  
[7,8,8,9,5,4],  
[3,8,8,6,5,4],  
[6,6,7,9,7,4],  
[3,8,8,6,5,4],  
[1,8,8,6,5,4],  
])

y_data = torch.FloatTensor([  
[0],  
[1],  
[1],  
[0],  
[1]  
])

In [12]:
print(x_data.shape) # m=5 d=6
print(y_data.shape) #

torch.Size([5, 6])
torch.Size([5, 1])


### Computing the Hypothesis

Pytorch has a torch.exp() function that resembles the exponential function.

In [13]:
print('e^1 equals :', torch.exp(torch.FloatTensor([1])))

e^1 equals : tensor([2.7183])


In [23]:
W = torch.randn((6,1), requires_grad=True)
W

tensor([[ 0.4224],
        [ 0.2673],
        [-0.4212],
        [-0.5107],
        [-1.5727],
        [-0.1232]], requires_grad=True)

In [24]:
b = torch.randn(1, requires_grad=True)
b

tensor([3.5870], requires_grad=True)

In [25]:
hypothesis = 1 / (1 + torch.exp(-(x_data.matmul(W) + b)))

In [26]:
hypothesis

tensor([[4.8072e-04],
        [4.1067e-04],
        [1.2116e-05],
        [4.1067e-04],
        [1.7648e-04]], grad_fn=<MulBackward0>)

torch 에서는 torch.sigmoid를 제공하기 때문에 위의 계산을 구지 할 필요가 없다

In [27]:
print(torch.sigmoid(torch.FloatTensor([1])))

tensor([0.7311])


In [29]:
hypothesis = torch.sigmoid(x_data.matmul(W)+b)

In [32]:
print(hypothesis)
print(hypothesis.shape)

tensor([[4.8072e-04],
        [4.1067e-04],
        [1.2116e-05],
        [4.1067e-04],
        [1.7648e-04]], grad_fn=<SigmoidBackward>)
torch.Size([5, 1])


### Compute the Cost Function

지금까지 H(x)를 구하는 과정을 살펴 보았습니다.  
그럼 우리는 이 값과 y_data의 차이를 살펴보고 이를 통해 cost를 구할 수 있습니다.  

In [34]:
losses = -(y_data * torch.log(hypothesis) + 
           (1 - y_data) * torch.log(1-hypothesis))

In [35]:
print(losses)

tensor([[4.8083e-04],
        [7.7977e+00],
        [1.1321e+01],
        [4.1076e-04],
        [8.6423e+00]], grad_fn=<NegBackward>)


In [36]:
cost = losses.mean() #평균을 취해줍니다.
print(cost)

tensor(5.5524, grad_fn=<MeanBackward0>)


##### 지금까지 위에 식을  torch.nn.functional의 binary_cross_entropy를 이용하면 해결이 가능합니다. 

- cross entropy 인데 binary한 경우
- cross entropy란? 

In [37]:
F.binary_cross_entropy(hypothesis,y_data) #bce

tensor(5.5524, grad_fn=<BinaryCrossEntropyBackward>)

# Whole Training Procedure

전체 코드를 통해 학습을 해 보자.

In [54]:
# Training Data
x_train = torch.FloatTensor([  
[7,8,8,9,5,4],  
[3,8,4,6,5,0],  
[6,6,7,6,7,4],  
[3,8,2,1,6,2],  
[1,8,2,6,6,4],  
])

y_train = torch.FloatTensor([  
[0],  
[1],  
[1],  
[0],  
[1]  
])

# 모델 초기화
W = torch.randn((6,1), requires_grad=True)
b = torch.randn(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.3) #SGC를 이용해서 W,b를 학습할꺼다
nb_epochs = 1000
for epoch in range(nb_epochs) :
    hypothesis = torch.sigmoid(x_train.matmul(W)+b)
    cost =  F.binary_cross_entropy(hypothesis,y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() #혹시라도 optimizer가 grad를 구한게 있으면 0로 초기화 한다
    cost.backward() # backpropagation을 수행
    optimizer.step() # cost값을 minimize 하는 방법으로 수행합니다.
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch,nb_epochs,cost.item()))
        

Epoch    0/1000 Cost: 4.426760
Epoch  100/1000 Cost: 2.220122
Epoch  200/1000 Cost: 0.001048
Epoch  300/1000 Cost: 0.000995
Epoch  400/1000 Cost: 0.000959
Epoch  500/1000 Cost: 0.000932
Epoch  600/1000 Cost: 0.000909
Epoch  700/1000 Cost: 0.000889
Epoch  800/1000 Cost: 0.000872
Epoch  900/1000 Cost: 0.000855


In [58]:
W

tensor([[-3.0737],
        [-3.8741],
        [-0.4423],
        [ 3.2403],
        [ 5.9475],
        [-2.3517]], requires_grad=True)

### Evaluation 성능평가!!

In [71]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)
print(hypothesis[:3])

tensor([[0.0012],
        [0.9991],
        [0.9988]], grad_fn=<SliceBackward>)


자 그럼 위의 값들을 가지고 0 또는1 찍어야 한다.

We can change hypothesis (real number from 0 to 1) to binary prediction (either 0 or 1) by comparing them to 0.5

In [73]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction)

"""
y_train = torch.FloatTensor([  
[0],  
[1],  
[1],  
[0],  
[1]  
])
"""

tensor([[False],
        [ True],
        [ True],
        [False],
        [ True]])


'\ny_train = torch.FloatTensor([  \n[0],  \n[1],  \n[1],  \n[0],  \n[1]  \n])\n'

로 잘 구해진것을 볼 수 있습니다.그러나 자료(sample)이 엄청 많다면 이렇게 하나하나 비교하는게 힘들수 있습니다,  
   
그래서 우리는 ByteTensor값인 prediction을 .float()으로 변환하여 정답과 직접 비교 해보도록 하겠습니다. 

In [74]:
correct_prediction = prediction.float() == y_train
print(correct_prediction)

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


# Higher Implementataion with Class

위의 방법도 좋지만 실전에선 아래와 같은방법으로 데이터 학습을 할 것입니다.

In [79]:
class BinaryClassifier(nn.Module) : ##nn.Module이라는 추상클래스를 상속 받습니다.
    def __init__(self) :
        super().__init__()
        self.linear = nn.Linear(6,1) # W,b가 들어있는 linear layer를 선언
                                     # 위와같이 선언하면 W는 (8,1) 크기 b (1,) 를 나타낸다.
                                     # 이렇게만 해줘도 d의 차원은 알 수 있다 d=8 , m은 ?? 아직,,
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x) :            # 아래 x가 self.linear(x)를 통과하고 나서 self.sigmoid에 넣어주면 이것이 전체 foward를 의미합니다.
        return self.sigmoid(self.linear(x))

In [80]:
model = BinaryClassifier()

In [82]:
# Training Data
x_train = torch.FloatTensor([  
[7,8,8,9,5,4],  
[3,8,4,6,5,0],  
[6,6,7,6,7,4],  
[3,8,2,1,6,2],  
[1,8,2,6,6,4],  
])

y_train = torch.FloatTensor([  
[0],  
[1],  
[1],  
[0],  
[1]  
])

optimizer = optim.SGD(model.parameters(), lr=0.3) 
"""
위와 다르게 W,b가 없다!?
"""

nb_epochs = 1000
for epoch in range(nb_epochs) :
    hypothesis = model(x_train)
    cost =  F.binary_cross_entropy(hypothesis,y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() #혹시라도 optimizer가 grad를 구한게 있으면 0로 초기화 한다
    cost.backward() # backpropagation을 수행
    optimizer.step() # cost값을 minimize 하는 방법으로 수행합니다.
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch,nb_epochs,cost.item()))
        

Epoch    0/1000 Cost: 1.586920
Epoch  100/1000 Cost: 5.526752
Epoch  200/1000 Cost: 5.526547
Epoch  300/1000 Cost: 5.526697
Epoch  400/1000 Cost: 5.532208
Epoch  500/1000 Cost: 5.528711
Epoch  600/1000 Cost: 0.003727
Epoch  700/1000 Cost: 0.001326
Epoch  800/1000 Cost: 0.001249
Epoch  900/1000 Cost: 0.001199


In [84]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)
prediction = hypothesis >= torch.FloatTensor([0.5])
correct_prediction = prediction.float() == y_train
print(correct_prediction)

tensor([[True],
        [True],
        [True],
        [True],
        [True]])
